In [6]:
import riak

client = riak.RiakClient()

In [7]:
client.ping()

True

In [8]:
bucket = client.bucket('rooms')

In [9]:
STYLES = ["single", "double", "queen", "king","suite"]

In [17]:
from random import randint, seed

seed(10)

for floor in range(1,100):
    current_rooms_block = floor * 100
    print "Making rooms #{} - #{}".format(current_rooms_block, current_rooms_block + 100)
    
    # Put 100 rooms on each floor (huge hotel!)
    for room in range(1,100):
        # Create a unique room number as the key
        key = str(current_rooms_block + room)
        ro = riak.RiakObject(client, bucket, key)
        
        style = STYLES[randint(0, len(STYLES)-1)]
        capacity = randint(0, 8) + 1
              
        ro.content_type = "application/json"
        ro.data = {'style': style, 'capacity': capacity}
        ro.store()
    

Making rooms #100 - #200
Making rooms #200 - #300
Making rooms #300 - #400
Making rooms #400 - #500
Making rooms #500 - #600
Making rooms #600 - #700
Making rooms #700 - #800
Making rooms #800 - #900
Making rooms #900 - #1000
Making rooms #1000 - #1100
Making rooms #1100 - #1200
Making rooms #1200 - #1300
Making rooms #1300 - #1400
Making rooms #1400 - #1500
Making rooms #1500 - #1600
Making rooms #1600 - #1700
Making rooms #1700 - #1800
Making rooms #1800 - #1900
Making rooms #1900 - #2000
Making rooms #2000 - #2100
Making rooms #2100 - #2200
Making rooms #2200 - #2300
Making rooms #2300 - #2400
Making rooms #2400 - #2500
Making rooms #2500 - #2600
Making rooms #2600 - #2700
Making rooms #2700 - #2800
Making rooms #2800 - #2900
Making rooms #2900 - #3000
Making rooms #3000 - #3100
Making rooms #3100 - #3200
Making rooms #3200 - #3300
Making rooms #3300 - #3400
Making rooms #3400 - #3500
Making rooms #3500 - #3600
Making rooms #3600 - #3700
Making rooms #3700 - #3800
Making rooms #3800

In [66]:
%%file ~/notebooks/riak/input.json
{
    "inputs":[
        ["rooms","101"],["rooms","102"],["rooms","103"]
    ],
    "query":[
        {"map":{
            "language":"javascript",
            "source":
                "function(v) {
                    /* From the Riak object, pull data and parse it as JSON */
                    var parsed_data = JSON.parse(v.values[0].data);
                    var data = {};
                    /* Key capacity number by room style string */
                    data[parsed_data.style] = parsed_data.capacity;
                    return [data];
                    }"
        }}
    ]
}

Overwriting /home/learner/notebooks/riak/input.json


In [67]:
!curl -s -X POST -H "content-type:application/json" http://localhost:8098/mapred \
        --data "@/home/learner/notebooks/riak/input.json"

[{"queen":4},{"queen":2},{"suite":8}]

In [48]:
%%file ~/notebooks/riak/input.json
{
  "inputs":[
    ["rooms","101"],["rooms","102"],["rooms","103"]],
  "query":[
    {"map":{
      "language":"javascript",
      "source":
      "function(v) {
        var parsed_data = JSON.parse(v.values[0].data); 
        var data = {};
        var floor = ~~(parseInt(v.key) / 100);
        data[floor] = parsed_data.capacity; 
        return [data];
      }"}
  },
  {"reduce": {
      "language": "javascript",
      "source":
        "function(v) {
            var totals = {};
            for (var i in v) {
              for(var floor in v[i]) {
                if( totals[floor] ) 
                  totals[floor] += v[i][floor];
                else
                  totals[floor] = v[i][floor];
              } 
            }

            return [totals];
          }"}
  } ]
}

Overwriting /home/learner/notebooks/riak/input.json


In [49]:
!curl -s -X POST -H "content-type:application/json" http://localhost:8098/mapred \
        --data "@/home/learner/notebooks/riak/input.json"

[{"1":14}]

In [58]:
%%file ~/notebooks/riak/input.json
function(v) {
    var parsed_data = JSON.parse(v.values[0].data);
    var data = {};
    data[parsed_data.style] = parsed_data.capacity;
    return [data];
}

Overwriting /home/learner/notebooks/riak/input.json


In [59]:
!curl -X PUT -H "content-type:application/json" http://localhost:8098/riak/my_functions/map_capacity \
        --data "@/home/learner/notebooks/riak/input.json"

In [68]:
%%file ~/notebooks/riak/input.json
{
"inputs":[
["rooms","101"],["rooms","102"],["rooms","103"]
],
"query":[
{"map":{
"language":"javascript",
"bucket":"my_functions",
"key":"map_capacity"
}}
]
}

Overwriting /home/learner/notebooks/riak/input.json


In [70]:
!curl -X POST -H "content-type:application/json" \
http://localhost:8098/mapred --data "@/home/learner/notebooks/riak/input.json"

[{"queen":4},{"queen":2},{"suite":8}]

In [86]:
#"key_filters":[["string_to_int"], ["less_than", 1000]]

In [87]:
%%file ~/notebooks/riak/input.json
{
"inputs":{
    "bucket":"rooms",
    "key_filters": [["string_to_int"], ["between", 4200, 4399]]
},
"query":[
{"map":{
"language":"javascript",
"bucket":"my_functions",
"key":"map_capacity"
}},
{"reduce":{
"language":"javascript",
"source":
"function(v) {
var totals = {};
for (var i in v) {
for(var style in v[i]) {
if( totals[style] ) totals[style] += v[i][style];
else totals[style] = v[i][style];
}
}
return [totals];
}"
}}
]
}

Overwriting /home/learner/notebooks/riak/input.json


In [88]:
!curl -X POST -H "content-type:application/json" \
http://localhost:8098/mapred --data "@/home/learner/notebooks/riak/input.json"

[{"queen":201,"suite":209,"double":169,"single":228,"king":146}]